# 시맨틱 검색 - Semantic Search


## 시맨틱 검색 (Semantic Search)

오픈서치의 시맨틱 검색은 검색 쿼리의 의미를 이해하고, 그에 따라 가장 관련성 높은 결과를 사용자에게 제공하는 기능입니다. 이는 전통적인 키워드 기반 검색과 달리, 검색 쿼리의 맥락과 의미를 분석하여 보다 정확하고 관련성 높은 검색 결과를 제공합니다. 예를 들어 "아마존"이라는 단어가 상품명인지, 회사명인지, 아니면 지역명인지 등 쿼리의 의미를 파악하여 그에 맞는 검색 결과를 제공합니다.


## 신경망 검색(Neural Search)

수년 동안 고객들은 OpenSearch k-NN을 기반으로 시맨틱 검색 애플리케이션을 구축하기 위해서는 텍스트 임베딩 모델을 검색 및 수집 파이프라인에 통합하기 위해 추가적인 미들웨어를 구축해야 하는 부담이 있었습니다. 이제 Amazon SageMaker와 Amazon Bedrock와의 통합을 통해 신경망 검색을 강화하며 클러스터에서 실행되는 시맨틱 검색 파이프라인을 지원할 수 있습니다.

신경망 검색은 텍스트를 벡터로 변환하고 인덱싱 시간과 검색 시간 모두에서 벡터 검색을 용이하게 합니다. 인덱싱 중에 신경망 검색은 문서 텍스트를 벡터 임베딩으로 변환하고 텍스트와 그 벡터 임베딩을 모두 벡터 인덱스에 인덱싱합니다. 신경망 쿼리를 사용하는 경우 신경망 검색은 쿼리 텍스트를 벡터 임베딩으로 변환하고, 벡터 검색을 사용하여 쿼리와 문서 임베딩을 비교한 다음 가장 가까운 결과를 반환합니다.

문서를 인덱스에 인제스트하기 전에 문서는 기계 학습(ML) 모델을 통과하게 되며, 이 모델은 문서 필드에 대한 벡터 임베딩을 생성합니다. 검색 요청을 보내면 쿼리 텍스트나 이미지도 ML 모델을 통과하여 해당 벡터 임베딩을 생성합니다. 그런 다음 신경망 검색이 임베딩에 대한 벡터 검색을 수행하고 일치하는 문서를 반환합니다.

신경망 검색을 사용하면 OpenSearch API를 통해 인간의 언어로 검색 쿼리를 실행하고, Amazon SageMaker에서 호스팅되거나 Amazon Bedrock에서 관리하는 텍스트 임베딩을 통해 의미론적 이해와 유사성을 고려한 텍스트 임베딩을 사용하여 더 정확한 결과를 제공할 수 있습니다.


## 사전준비


필요한 패키지를 설치합니다.


In [2]:
!pip install -q boto3
!pip install -q requests
!pip install -q requests-aws4auth
!pip install -q opensearch-py
!pip install -q tqdm
!pip install -q boto3

키워드 검색 단계에서와 마찬가지로 데이터를 준비합니다.


In [63]:
import pandas as pd
import requests

df = pd.read_csv("./data/sample.csv", low_memory=False)
df.head(5)

,title,genre,year,date,rating,vote_count,plot,main_act,supp_act
0,변호인,드라마,2013,12.18,8.99,94574,"198 년대 초 부산. 빽 없고, 돈 없고, 가방끈도 짧은 세무 변호사 송우석(송강...",송강호|김영애|오달수|곽도원|임시완,송영창|정원중|조민기|이항나|이성민|차은재|차광수|한기중|심희섭|조완기
1,어벤져스: 엔드게임,액션|SF,2019,4.24,9.38,68923,인피니티 워 이후 절반만 살아남은 지구 마지막 희망이 된 어벤져스 먼저 떠난 그들을...,로버트 다우니 주니어|크리스 에반스|크리스 헴스워스|마크 러팔로|스칼렛 요한슨|제레...,베네딕트 컴버배치|조 샐다나|크리스 프랫|채드윅 보스만|톰 홀랜드|안소니 마키|기네...
2,명량,액션|드라마,2014,7.30,8.44,66953,"1597년 임진왜란 6년, 오랜 전쟁으로 인해 혼란이 극에 달한 조선. 무서운 속도...",최민식|류승룡|조진웅,진구|이정현|김명곤|권율|노민우|김태훈|오타니 료헤이|이승준|김강일|박보검|이해영|...
3,부산행,액션|스릴러,2016,7.20,8.00,59184,"정체불명의 바이러스가 전국으로 확산되고 대한민국 긴급재난경보령이 선포된 가운데, 열...",공유|정유미|마동석|김수안|김의성|최우식|안소희,최귀화|정석용|예수정|박명신|장혁진
4,신과함께-죄와 벌,판타지|드라마,2017,12.20,7.83,58124,"저승 법에 의하면, 모든 인간은 사후 49일 동안 7번의 재판을 거쳐야만 한다. 살...",하정우|차태현|주지훈|김향기|김동욱|마동석,오달수|임원희|디오|이준혁|예수정|장광|정해균|김수안|남일우|정지훈


데이터의 스키마를 확인합니다.


벡터 필드에 영화에 대한 전체적인 정보를 담기 위해 전체 컬럼을 모두 조합한 `text` 컬럼을 추가합니다. 이 `text` 필드는 이후 ingest pipeline에 의해 임베딩될 필드입니다.

In [65]:
import pandas as pd
import numpy as np
import json
import sqlite3

data_path = './data'
with open(f'{data_path}/tables.json', 'rb') as ofp:
    meta = json.load(ofp)
data = meta[0]

data = [i for i in meta if i['db_id'] == 'department_store']

data  = data[0]    
columns = data["column_names_original"]
col_df = pd.DataFrame(columns).iloc[1:]
col_df.rename(columns={0: 'table_idx', 1: 'col_name'}, inplace=True)
col_df

types_df = pd.DataFrame(data["column_types"]).iloc[1:]
types_df.rename(columns={0: 'type'}, inplace=True)
types_df

merged_col = pd.concat([col_df, types_df], axis=1)

In [66]:
tables_df = pd.DataFrame(data["table_names_original"])
tables_df.reset_index(inplace=True)
tables_df.columns = ['table_idx', 'table_name']

meta = pd.merge(tables_df, merged_col, on=['table_idx'])
meta = meta.drop(columns=['table_idx'])

In [19]:
meta.head(2)

,table_name,col_name,type
0,Addresses,address_id,number
1,Addresses,address_details,text


In [21]:
## table 단위로 row를 생성한다면 아래와 같이 늘리는 작업이 필요함 
## 그런데 단순 col name말고 type, description까지 생긴다고 보면 그렇게는 못할지도 
# import pandas as pd


# def create_text(row, max_len=509):
#     text = ""
#     for col, val in row.items():
#         text += f"{col}: {val},"
#     if len(text) > max_len:
#         text = text[:max_len] + "..."

#     # print(text.rstrip("\n"))
#     return text.rstrip()


# # Assuming your DataFrame is called 'df'
# df["text"] = df.apply(create_text, axis=1)
# df.head(10)

In [22]:
# 줄거리의 길이가 512가 넘는 레코드가 있는지 확인합니다
# def find_long_plot_items(df):
#     long_plot_items = df[df["text"].str.len() > 512]
#     return long_plot_items


# find_long_plot_items(df).count()

### OpenSearch 도메인에 연결


In [10]:
def get_cfn_outputs(stackname, cfn):
    outputs = {}
    for output in cfn.describe_stacks(StackName=stackname)["Stacks"][0]["Outputs"]:
        outputs[output["OutputKey"]] = output["OutputValue"]
    return outputs

In [11]:
import boto3, json


region_name = "us-west-2"

cfn = boto3.client("cloudformation", region_name)
kms = boto3.client("secretsmanager", region_name)

stackname = "opensearch-workshop"
cfn_outputs = get_cfn_outputs(stackname, cfn)

aos_credentials = json.loads(
    kms.get_secret_value(SecretId=cfn_outputs["OpenSearchSecret"])["SecretString"]
)

aos_host = cfn_outputs["OpenSearchDomainEndpoint"]
aos_host

'search-opensearch-workshop-ashew5agtjkgsyxprzgu2m2oua.us-west-2.es.amazonaws.com'

In [12]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

auth = (aos_credentials["username"], aos_credentials["password"])

aos_client = OpenSearch(
    hosts=[{"host": aos_host, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
)

## 데이터 임베딩을 위한 Ingest Pipeline 생성


모델 배포 과정을 참고하여 배포된 모델의 ID를 확인하고 아래와 같이 변수에 초기화합니다.


In [13]:
import requests

search_model = {"query": {"match": {"name": "OpenSearch-Cohere"}}, "size": 10}

response = requests.get(
    "https://" + aos_host + "/_plugins/_ml/models/_search", auth=auth, json=search_model
)
model_info = json.loads(response.text)
model_id = model_info["hits"]["hits"][0]["_id"]
model_id

'DTDeTJAB3Hj2edbFglKU'

## 인제스트 파이프라인 생성

영화의 줄거리 정보를 가진 특정 필드("plot")에 대한 벡터 임베딩을 생성하는 파이프라인을 설정합니다. 이러한 임베딩은 검색 인덱스 필드("vector_field")에 저장되며, 효율적인 유사성 검색 및 검색 작업에 사용될 수 있습니다.


In [14]:
pipeline = {
    "description": "Text to Sql Task - OpenSearch-cohere-060124084807",
    "processors": [
        {
            "text_embedding": {
                "model_id": model_id,
                "field_map": {
                    "text": "vector_field",
                },
            }
        }
    ],
}

pipeline_id = "text2sql_meta_data"
# aos_client.ingest.delete_pipeline(id=pipeline_id)
aos_client.ingest.put_pipeline(id=pipeline_id, body=pipeline)

{'acknowledged': True}

## 인덱스 생성

movie_semantic 인덱스를 생성합니다. 아래 세팅 및 맵핑 정보 중 중요한 것은 다음과 같습니다.

-   index.knn: KNN 검색을 위해 True로 설정합니다.
-   default_pipeline: 위 단계에서 생성한 pipeline_id를 제공합니다.
-   index.knn.space_type: 임베딩 벡터끼리의 유사도를 파악할 때 사용할 알고리즘을 cosinesimil로 지정합니다


In [23]:
meta.head(2)

,table_name,col_name,type
0,Addresses,address_id,number
1,Addresses,address_details,text


In [46]:
index_name = "rag_semantic_ver2"

# aos_client.indices.delete(index=index_name)

rag_semantic = {
    "settings": {
        "max_result_window": 15000,
        "analysis": {"analyzer": {"analysis-nori": {"type": "nori", "stopwords": "_korean_"}}},
        "index.knn": True,
        "default_pipeline": pipeline_id,
        "index.knn.space_type": "l2",
    },
    "mappings": {
        "properties": {
            "table_name": {
                "type": "text",
            },
            "col_name": {
                "type": "text",
            },
            "type": {
                "type": "text",
            },
            "vector_field": {
                "type": "knn_vector",
                "dimension": 1024,
                "method": {"name": "hnsw", "space_type": "l2", "engine": "faiss"},
                "store": True,
            },

        }
    },
}
aos_client.indices.create(index=index_name, body=rag_semantic)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'rag_semantic_ver2'}

## 데이터 인제스트

키워드 검색과 동일하게 데이터를 인제스트합니다. 위 단계에서 인덱스를 생성할 때 INGEST PIPELINE을 설정했기 때문에 직접 임베딩 모델을 호출하여 데이터를 벡터로 변환하지 않아도 됩니다. 단, 데이터가 인제스트될 때 임베딩 모델을 호출해야 하는 단계가 있기 때문에 parallel_bulk의 병렬도가 높으면 에러가 발생할 있습니다. 여기서는 `thread_count`와 `queue_size`를 각각 1로 낮춰줍니다. 이 단계가 완료되는데는 약 4~5분 정도 소요됩니다.


In [47]:
from tqdm import tqdm
from opensearchpy import helpers

json_data = meta.to_json(orient="records", lines=True)
docs = json_data.split("\n")[:-1]  # To remove the last empty line


def _generate_data():
    for doc in docs:
        yield {"_index": index_name, "_source": doc}


succeeded = []
failed = []
for success, item in helpers.parallel_bulk(
    aos_client, actions=_generate_data(), chunk_size=10, thread_count=1, queue_size=1
):
    if success:
        succeeded.append(item)
    else:
        failed.append(item)

데이터 인제스트가 잘 마무리되었는지 확인합니다.


In [48]:
# Refresh the index to make the changes visible
aos_client.indices.refresh(index=index_name)

count = aos_client.count(index=index_name)
print(count)

{'count': 56, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}}


# 키워드 검색과 시맨틱 검색 결과 비교


## 키워드 검색을 위한 함수 생성

키워드 단계에서 정의한 키워드 검색 함수와 동일한 함수를 생성합니다.


In [49]:
def keyword_search(query_text):
    query = {
        "size": 10,
        "_source": {"excludes": ["text", "vector_field"]},
        "query": {
            "multi_match": {
                "query": query_text,
                "fields": ["table_name", "col_name"],
            }
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_score"],
            hit["_source"]["table_name"],
            hit["_source"]["col_name"],
            hit["_source"]["type"],            
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result, columns=["_score", "table_name", "col_name", "type"]
    )
    display(query_result_df)

## 시맨팀 검색을 위한 함수 생성

뉴럴 검색을 위한 함수를 생성합니다. 주목해야할 부분은 다음과 같습니다.

1. **`"_source": {"excludes": ["vector_field"]}`**: 검색 결과에서 **`vector_field`** 필드를 제외한 모든 필드를 반환합니다. 이는 벡터 데이터가 크기 때문에 전송 비용을 줄이기 위함입니다.
2. **`"query": { ... }`**: 실제 검색 쿼리를 정의합니다.
3. **`"neural": { ... }`**: 벡터 검색을 수행하기 위한 쿼리 유형입니다.
4. **`"vector_field": "vector_field_name"`**: 벡터 데이터가 저장된 필드 이름입니다.
5. **`"query_text": query_text`**: 검색할 텍스트 쿼리입니다.
6. **`"model_id": model_id`**: 벡터 임베딩을 생성하는 데 사용된 모델의 ID입니다.


In [54]:
def semantic_search(query_text):
    query = {
        "size": 10,
        "_source": {"excludes": ["text", "vector_field"]},
        "query": {
            "neural": {"vector_field": {"query_text": query_text, "model_id": model_id, "k": 10}},
        },
    }

    res = aos_client.search(index=index_name, body=query)

    query_result = []
    for hit in res["hits"]["hits"]:
        row = [
            hit["_score"],
            hit["_source"]["table_name"],
            hit["_source"]["col_name"],
            hit["_source"]["type"],            
        ]
        query_result.append(row)

    query_result_df = pd.DataFrame(
        data=query_result, columns=["_score", "table_name", "col_name", "type"]
    )
    display(query_result_df)

## 결과 비교

자연어 기반의 쿼리를 작성하고 키워드 검색과 시맨틱 검색의 결과를 비교해봅니다


In [58]:
query_text = "Product_Suppliers"

In [59]:
keyword_search(query_text)

,_score,table_name,col_name,type
0,2.367124,Product_Suppliers,product_id,number
1,2.159484,Product_Suppliers,date_supplied_from,time
2,2.079442,Product_Suppliers,supplier_id,number
3,1.791759,Product_Suppliers,total_amount_purchased,text
4,1.481604,Product_Suppliers,date_supplied_to,time
5,1.481604,Product_Suppliers,total_value_purchased,number


In [60]:
semantic_search(query_text)
# print(index_name)

,_score,table_name,col_name,type


- semantic search는 아무것도 나오지 않는다. 

## 단순히 테이블을 때려 박는 게 아니라, 테이블에 대한 정보또한 llm으로 생성할 필요가 있다.